In [18]:
from urllib.parse import quote_plus
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

In [19]:
def connection(hostname, username, password, dbname):
    con = None
    password = quote_plus(password)
    try:
        con = mysql.connector.connect(hostname,username,password,dbname)
        print("Connection successful!")
    except Error as e: # type: ignore
        print(f"The error {e} has occured.")
    return con

def engine(hostname, username, password, dbname, port):
    eng = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}:{port}/{dbname}')
    return eng

In [21]:
def csv_to_staging(datafile):
    eng = engine("localhost", "root", "Layaldbroot1997", "feedback_source", "3306")
    df = pd.read_csv(datafile, sep=', ', engine='python')
    #print(df.head())
    df.to_sql(name="feedback",con=eng, schema="feedback_source",if_exists="replace",index = False)

csv_to_staging("sentiment-analysis.csv")

def extract_source():
    conn = connection("localhost", "root", "Layaldbroot1997", "feedback_source")
    cursor = conn.cursor()
    query = "Select * from feedback_source.feedback"
    cursor.execute(query)
    data = cursor.fetchall()
    column_names = [i[0] for i in cursor.description]
    df = pd.DataFrame(data, columns=column_names)
    return df

In [ ]:
def transform_feedback():
    df = extract_source()

    # Take a copy of the df
    df = df.copy()

    # Drop dupicates
    df =  df.drop_duplicates()

    # Rename columns properly
    df.rename({'"Text': 'Text', 'Confidence Score"': 'Confidence Score'}, inplace=True)

    # Remove characters from values
    df.loc[:,'"Text'] = df['"Text'].str.replace(r'\"', '', regex=True)
    df.loc[:,'Confidence Score"'] = df['Confidence Score"'].str.replace(r'\"', '', regex=True)
    df.loc[:,'User ID'] = df['User ID'].str.replace(r'@', '', regex=True)

    return df